In [1]:
import requests
import pandas as pd

url = "https://zillow-com1.p.rapidapi.com/property"

querystring = {"zpid":"112156606"}

headers = {
	"x-rapidapi-key": "287c0c69cbmsh41c226a35114322p1022a8jsn65498208884c",
	"x-rapidapi-host": "zillow-com1.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

# Convert the response to a pandas dataframe
data = response.json()

# The data returned from Zillow's API is in nested JSON format.
# To convert it into a flat dataframe, we need to flatten the dictionary.
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

flat_data = flatten_json(data)
df = pd.DataFrame([flat_data])

# Get all column names from the pandas dataframe and output them into a csv file
column_names = list(df.columns)
pd.DataFrame(column_names).to_csv("zillowattributes.csv", index=False, header=["Column Name"])

shortdf = df[['zpid', 'streetAddress', 'monthlyHoaFee', 'yearBuilt', 'latitude', 'longitude', 'livingAreaValue','climate_floodSources_primary_insuranceRecommendation','climate_floodSources_primary_riskScore_label', 'climate_floodSources_primary_riskScore_value', 'rentZestimate', 'propertyTaxRate', 'timeOnZillow', 'url', 'zestimate', 'bedrooms', 'bathrooms', 'zipcode', 'price', 'homeStatus']]
print(shortdf)

# Import the gspread library to interact with Google Sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Define the scope of permissions we need for our script to access Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# Authenticate and create a client using our credentials file
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

# Open the Google Sheet by its URL
sheet_url = "https://docs.google.com/spreadsheets/d/1w4suUrGjIhfn_ufYhHJqgYE9V32GEvoCSBgcanjdwms/edit?gid=0#gid=0"
spreadsheet = client.open_by_url(sheet_url)

# Select the first worksheet in the spreadsheet
worksheet = spreadsheet.sheet1

# Get all values from the worksheet and store them in a pandas dataframe, pass expected_headers to avoid GSpreadException
expected_headers = shortdf.columns.tolist()
existing_data = pd.DataFrame(worksheet.get_all_records(expected_headers=expected_headers))

# Check if the 'zpid' column exists in the existing data before checking its values
if 'zpid' not in existing_data.columns:
    # Append the new data to the worksheet if there is no zpid column
    shortdf = shortdf.astype(str)  # Convert all columns to string type to avoid JSON serialization errors
    worksheet.append_rows([shortdf.iloc[0].tolist()], value_input_option='USER_ENTERED')
else:
    # Check if the home already exists in the dataframe by checking if its zpid matches any existing zpids in the sheet
    shortdf['zpid'] = shortdf['zpid'].astype(str)  # Convert zpid column to string type for comparison
    existing_data['zpid'] = existing_data['zpid'].astype(str)
    if shortdf['zpid'].values[0] not in existing_data['zpid'].values:
        # Append the new data to the worksheet if it does not exist
        shortdf = shortdf.astype(str)  # Convert all columns to string type to avoid JSON serialization errors
        worksheet.append_rows([shortdf.iloc[0].tolist()], value_input_option='USER_ENTERED')



        zpid       streetAddress monthlyHoaFee  yearBuilt   latitude  \
0  112156606  6319 Riverview Way          None       2009  29.757406   

   longitude  livingAreaValue  \
0  -95.49642             2149   

  climate_floodSources_primary_insuranceRecommendation  \
0                                       NOT_CRITICAL     

  climate_floodSources_primary_riskScore_label  \
0                                      MINIMAL   

   climate_floodSources_primary_riskScore_value  rentZestimate  \
0                                             1           2669   

   propertyTaxRate timeOnZillow  \
0              1.6        1 day   

                                                 url  zestimate  bedrooms  \
0  /homedetails/6319-Riverview-Way-Houston-TX-770...     538700         3   

   bathrooms zipcode   price homeStatus  
0          3   77057  550000   FOR_SALE  


C:\Users\kaysh\AppData\Local\Temp\ipykernel_66704\1729117313.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shortdf['zpid'] = shortdf['zpid'].astype(str)  # Convert zpid column to string type for comparison
